## Read d3plot from LS-DYNA using lasso-python

In [ ]:
from read_d3plot import *

path_to_d3plot = r'C:\Users\272766h\Curtin University of Technology Australia\Zitong Wang - Data generation\C30_120mm\0.4_10\d3plot'
d3plot = D3plot(path_to_d3plot)

In [ ]:
def extract_trajectory_type_strain(d3plot):
    """Read particle (element) trajectory of coordinates.
    
    Input: the d3plot data
    Output: particle_trajectories of shape [ntimesteps, nparticles, 3]
    """
    
    ## node_displacement is actually node coords in all steps, shape [nstep, nnodes, 3]
    node_trajectories = d3plot.arrays["node_displacement"]
    ## Each solid element (cubic) is defined by 8 nodes
    # element_solid_node_indexes = d3plot.arrays["element_solid_node_indexes"]
    ## each beam involves 2 nodes, but the array shows 5 with 3rd being the same as 2nd
    ## and 4th, 5th looks unrelated
    ## Using LS-PrePost outputs 3 nodes per beam, with 3rd also being the same as 2nd
    ## Therefore, only the first 2 nodes are used
    element_beam_node_indexes = d3plot.arrays["element_beam_node_indexes"][:, :2]
    
    # Convert the solid node indexes to a set for quick look-up
    sph_node_indexes = d3plot.arrays["sph_node_indexes"]
    SPH_trajectories = node_trajectories[:, sph_node_indexes, :]

    element_beam_node_indexes = np.unique(element_beam_node_indexes)
    
    element_beam_trajectories = node_trajectories[:, element_beam_node_indexes, :]

    particle_trajectories = np.concatenate((SPH_trajectories, element_beam_trajectories), axis=1)

    # Derive particle types, 0 concrete, 1 rebar, 2 boundary
    # boundary is always 150 mm on the two ends of y-axis
    SPH_types = np.zeros(SPH_trajectories.shape[1])
    beam_types = np.ones(element_beam_trajectories.shape[1])
    particle_type = np.concatenate((SPH_types, beam_types), axis=0)
    LEFT_BOUNDARY = -855    # particle_trajectories[0, :, 1].min() + 150, this not aligned with the data from txt
    RIGHT_BOUNDARY = 855    # particle_trajectories[0, :, 1].max() - 150
    mask = (particle_trajectories[0, :, 1] >= RIGHT_BOUNDARY) | (particle_trajectories[0, :, 1] <= LEFT_BOUNDARY)
    particle_type[mask] = 2
    
    # Strain
    solid_eps = d3plot.arrays["element_solid_effective_plastic_strain"][:, :, 0]
    beam_eps = np.zeros((solid_eps.shape[0], element_beam_trajectories.shape[1]))
    particle_strains = np.concatenate((solid_eps, beam_eps), axis=1)
    
    return particle_trajectories, particle_type, particle_strains

particle_trajectories, particle_type, particle_strains = extract_trajectory_type_strain(d3plot)

print(particle_trajectories.shape)
print(particle_strains.shape)
print(particle_type.shape)

## Enforce monotonical increasing EPS

In [ ]:
import numpy as np


def enforce_eps_non_decreasing(particle_strains):
    # Compute the differences between adjacent time steps
    strains_diff = np.diff(particle_strains, axis=0)

    # Set any negative differences to zero
    strains_diff[strains_diff < 0] = 0

    # Reconstruct the corrected strains using cumulative sum,
    # starting with the initial strain values
    corrected_strains = np.concatenate((particle_strains[:1, :], strains_diff), axis=0).cumsum(axis=0)
    return corrected_strains

particle_strains = enforce_eps_non_decreasing(particle_strains)

## Downsampling data
- The provided data in d3plot should have timestep=0.01ms
- FGN subsample that with a factor of 6, resulting in timestep=0.06ms
- FGN requires 10 steps (or 0.6ms) to initilise
- Therefor, given whatever data (>0.6ms) we need to downsample first and then extract the last 10 steps
- CAUTION: EPS must be converted to monotonically increasing EPS before downsampling. Otherwise it results in peak missing
- That's because the element erosion will result in element_eps=0 after reaching threshold

In [ ]:
INPUT_SEQUENCE_LENGTH = 10
STEP_SIZE = 6

def timestep_downsample(particle_trajectories, particle_strains):
    particle_trajectories_downsampled = particle_trajectories[::STEP_SIZE]
    particle_trajectories_downsampled = particle_trajectories_downsampled[-INPUT_SEQUENCE_LENGTH:]
    
    particle_strains_downsampled = particle_strains[::STEP_SIZE]
    # Convert EPS to ResEPS (residual eps)
    strains_diff = np.diff(particle_strains_downsampled, axis=0)
    ## The initial strain should be the strain at step=-10 after downsampling
    init_strains = particle_strains_downsampled[-INPUT_SEQUENCE_LENGTH:-INPUT_SEQUENCE_LENGTH+1, :]
    ## The final ResEPS should combine the init_strain and following 9 steps of strain_dff
    particle_strains_downsampled = np.concatenate((init_strains, strains_diff[-INPUT_SEQUENCE_LENGTH+1:]), axis=0)
    
    return particle_trajectories_downsampled, particle_strains_downsampled

particle_trajectories, particle_resStrains = timestep_downsample(particle_trajectories, particle_strains)
print(particle_trajectories.shape, particle_resStrains.shape)                                                                                                

## Check data integrity

In [ ]:
def check_data_integrity(particle_trajectories, particle_strains, particle_type):
    nstep, nparticles, dim = particle_trajectories.shape
    
    # Shape check:
    assert particle_type.shape == (nparticles,), "Mismatch in shapes: particle_type"
    assert particle_strains.shape == (nstep, nparticles), "Mismatch in shapes: particle_strains"
    assert particle_trajectories.shape[0] == particle_strains.shape[0] == 10, "INPUT_SEQUENCE_LENGTH is not 10"
    
    # Missing value check
    for array, name in zip([particle_trajectories, particle_type, particle_strains],
                           ["particle_trajectories", "particle_type", "particle_strains"]):
        if np.any(np.isnan(array)):
            print(f"Missing values detected in {name}")
    
    # Value Range CHecks:
    for i, axis in enumerate(["x", "y", "z"]):
        print(f"{axis}-axis min: {np.min(particle_trajectories[:, :, i]):.2f}, max: {np.max(particle_trajectories[:, :, i]):.2f}")
    
    if np.any((particle_strains < 0) | (particle_strains > 2)):
        print("Effective Plastic strain values out of range [0, 2]")
        

check_data_integrity(particle_trajectories, particle_resStrains, particle_type)

## Test data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


data = particle_ResStrains
nstep = data.shape[0]
nnode = data.shape[1]

# Randomly pick 20 nodes
random_nodes = np.random.choice(particle_strains.shape[1], 30, replace=False)

# Initialize the plot with subplots in 2 or 3 rows
nrows = 3  # You can change this to 2 if you prefer
ncols = int(np.ceil(30 / nrows))
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 7))

# Flatten the axes array for easy indexing
axes = axes.flatten()

# Plot the eps history for the selected nodes
for i, node in enumerate(random_nodes):
    ax = axes[i]
    ax.plot(data[:, node])
    ax.set_title(f'Node {node}')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Eps')

# Remove any unused subplots
for i in range(len(random_nodes), nrows * ncols):
    fig.delaxes(axes[i])

# Add a main title
plt.suptitle('Eps History for Randomly Selected Nodes')

# Show the plot
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()